# Brownian Motion (fixed step vs. adaptive)

In [ ]:
using KadanoffBaym
using LinearAlgebra

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

In [ ]:
T = 0.5
theta = 1.
t_scale = (iszero(theta) ? one(theta) : abs(theta))

N_0 = [1.]
Ds = [8.];

## Adaptive

In [ ]:
function solution(N_0::Float64, D::Float64)
    F_ana(t1, t2) =(F[1, 1] - D/(2 * theta)) * exp(-theta * (t1 + t2)) + D/(2 * theta) * exp(-theta * abs(t1 - t2))

    f_vert(_, _, t1, t2) = [-theta * F[t1, t2]]
    f_diag(_, times, t)  = (println(times[t]); [-2theta * F[t, t] + D])

    F = GreenFunction(N_0 * ones(1,1), Symmetrical)
    
    sol = kbsolve(f_vert, f_diag, [F], (0.0, T), atol=0, rtol=1e-4, dtini=0.5/2^8, kmax=4)

    return [sol.t, F.data, [F_ana(t1, t2) for t1 in sol.t, t2 in sol.t]]
end;

In [ ]:
s = mapreduce(solution, hcat, N_0, Ds);

In [ ]:
idx = 1
dts = s[1, idx][2:end] .- s[1, idx][1:end-1];

### Plotting

In [ ]:
figure(figsize=(5, 3))
semilogy(s[1, idx][2:end], dts, "o-")
xlim(0, theta * T)
ylim(1e-3, 1e-1)
xlabel("\$\\theta t\$")
ylabel("\$\\mathrm{d} t\$")
tight_layout(pad=0.25, w_pad=1, h_pad=0)
# savefig("brownian_motion_adaptive_dt.pdf")

In [ ]:
cmap = "gist_heat";
colors = ["C0", "C1", "C2"];
lss = ["-", "--", "-."];

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
for k in eachindex(N_0)
    plot(s[1,k], diag(s[2,k]), label="\$xxx\$", lw=1.5, c=colors[k], ls=lss[k])
end
xlim(0, theta * T)
ylim(0, 1.1 * Ds[1]/(2theta))
yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\theta t\$")
ylabel("\$F(t, t)\$")

ax = subplot(122)
plot([], [], label="\$D/\\theta\$", c="w")
for k in eachindex(N_0)
    semilogy(s[1,k], abs.(diag(s[2,k])  .- diag(s[3,k])), lw=1.5, c=colors[k], label="\$"*string(Ds[k]/(theta))*"\$", ls=lss[k])
end

xlim(0, theta * T)
# ylim(1e-6, 1e-2)
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\theta t\$")
ylabel("\$\\left|F(t, t) - \\mathcal{F}(t, t)\\right|\$", labelpad=16)
ax.yaxis.set_label_position("right")
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
# # ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.25, w_pad=1, h_pad=0)

## Fixed

In [ ]:
function solution(N_0::Float64, D::Float64)
    F_ana(t1, t2) =(F[1, 1] - D/(2 * theta)) * exp(-theta * (t1 + t2)) + D/(2 * theta) * exp(-theta * abs(t1 - t2))

    f_vert(_, _, t1, t2) = [-theta * F[t1, t2]]
    f_diag(_, times, t)  = (println(times[t]); [-2theta * F[t, t] + D])

    F = GreenFunction(N_0 * ones(1,1), Symmetrical)
    
    sol = kbsolve(f_vert, f_diag, [F], (0.0, T), atol=0, rtol=1e0, dtmax=0.5/2^8, dtini=0.5/2^8, kmax=4, γ=1)

    return [sol.t, F.data, [F_ana(t1, t2) for t1 in sol.t, t2 in sol.t]]
end;

In [ ]:
s = mapreduce(solution, hcat, N_0, Ds);

In [ ]:
idx = 1
dts = s[1, idx][2:end] .- s[1, idx][1:end-1];

In [ ]:
figure(figsize=(5, 3))
semilogy(s[1, idx][2:end], dts, "o-")
xlim(0, theta * T)
ylim(1e-3, 1e-1)
xlabel("\$\\theta t\$")
ylabel("\$\\mathrm{d} t\$")
tight_layout(pad=0.25, w_pad=1, h_pad=0)
# savefig("brownian_motion_fixed_dt.pdf")

### Plotting

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
for k in eachindex(N_0)
    plot(s[1,k], diag(s[2,k]), label="\$xxx\$", lw=1.5, c=colors[k], ls=lss[k])
#     plot(s[1,k], s[3,k], ls="--", lw=4.0, alpha=0.4, c=colors[k])
end
xlim(0, theta * T)
ylim(0, 1.1 * Ds[1]/(2theta))
yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\theta t\$")
ylabel("\$F(t, t)\$")

ax = subplot(122)
plot([], [], label="\$D/\\theta\$", c="w")
for k in eachindex(N_0)
    semilogy(s[1,k], abs.(diag(s[2,k])  .- diag(s[3,k])), lw=1.5, c=colors[k], label="\$"*string(Ds[k]/(theta))*"\$", ls=lss[k])
end

xlim(0, theta * T)
# ylim(1e-6, 1e-2)
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\theta t\$")
ylabel("\$\\left|F(t, t) - \\mathcal{F}(t, t)\\right|\$", labelpad=16)
ax.yaxis.set_label_position("right")
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
# # ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.25, w_pad=1, h_pad=0)